In [41]:
import torch
import torch.utils.data
import pandas as pd
import numpy as np
import starter_code.utils as starter
import starter_code.visualize as vis
import os
import h5py
import tqdm
from src.unet import UNet3D

In [43]:
net = UNet3D(1, 3, False)

In [161]:
t = torch.randn((8, 1, 8, 8, 8))

In [162]:
out = net(t)

In [163]:
from sklearn.metrics import confusion_matrix

In [164]:
batch_prediction = torch.argmax(torch.softmax(out, 1), 1, keepdim=True)

In [165]:
gt = torch.randint(3, (8, 1, 8, 8, 8))

In [190]:
batch_prediction[0:1,:].shape

torch.Size([1, 1, 8, 8, 8])

In [169]:
def dice_score(tp, fp, fn):
    return (2 * tp)/(2 * tp + fp + fn)

In [170]:
def get_class_stats(target_class, cm):
    tp = cm[target_class, target_class]
    fp = np.sum(cm[:, target_class]) - tp
    fn = np.sum(cm[target_class, :]) - tp
    return (tp, fp, fn)

In [183]:
def score_function(prediction, ground_truth):
    pred_flat = prediction.view(-1)
    gt_flat = ground_truth.view(-1)
    cm = confusion_matrix(gt_flat, pred_flat)
    tp, fp, fn = get_class_stats(1, cm)
    kidney_score = dice_score(tp, fp, fn)
    tp, fp, fn = get_class_stats(2, cm)
    tumor_score = dice_score(tp, fp, fn)
    return (kidney_score + tumor_score) / 2

In [195]:
score_function(batch_prediction, gt)

0.35913340916927927

In [192]:
scores = [score_function(batch_prediction[i-1:i, :], gt[i-1:i, :]) for i in range(1, 8)]

In [194]:
np.mean(scores)

0.35416652986042